### 使用docs/example_seisbench.ipynb 以及 GaMMA/docs/example_phasenet.ipynb
### set configurations

In [29]:
import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from pyproj import CRS, Transformer, Proj
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
from gamma.utils import association
import pandas as pd
from gamma.utils import from_seconds,association, estimate_eps
import numpy as np
import os
from pyproj import Proj
import time





## 换个新的试试看 参考GAMMA.ipynb

In [30]:
## read picks
picks = pd.read_csv('D:\MyRepository\Qiaojia\GammaAssociation\merged_pick_scaled.csv', parse_dates=["phase_time"], usecols=["station_id", "phase_time", "phase_type", "phase_score", "phase_amplitude"])     # parse_dates用于讲某列解析为日期格式
picks.rename(columns={"station_id": "id", "phase_time": "timestamp", "phase_type": "type", "phase_score": "prob", "phase_amplitude": "amp"}, inplace=True)
#picks = picks[:50000]
picks["time_idx"] = picks["timestamp"].apply(lambda x: x.strftime("%Y-%m-%d")) ## process by days
print("Pick format:", picks.iloc[:5])
print(picks.columns)



result_path = "D:/MyRepository/Qiaojia/AssResults_modifydepth"
if os.path.exists(result_path) == False:
    os.makedirs(result_path)
## Automatic region; you can also specify a region
# 可以自定义
# 台站的中心位置，经度最大最小值，纬度最大最小值

config = {}
config["ncpu"] = 19
config["center"] = (103.0,26.8)
config["xlim_degree"] = (102, 104)
config["ylim_degree"] = (26.5, 27.5)
config['degree2km'] = 111

config["x(km)"] = (np.array(config["xlim_degree"])-np.array(config["center"][0]))*config["degree2km"]
config["y(km)"] = (np.array(config["ylim_degree"])-np.array(config["center"][1]))*config["degree2km"]
config["z(km)"] = (0, 30)

## read stations
stations = pd.read_csv('D:\MyRepository\Qiaojia\GammaAssociation\宽频带布点.txt',sep=' ')
stations.rename(columns={"station_id": "id"}, inplace=True)
stations["x(km)"] = stations["longitude"].apply(lambda x: (x - config["center"][0])*config["degree2km"])
stations["y(km)"] = stations["latitude"].apply(lambda x: (x - config["center"][1])*config["degree2km"])
stations["z(km)"] = stations["elevation_m"].apply(lambda x: x/1e3)
print("Station format:", stations.iloc[:5])

### setting GMMA configs
config["dims"] = ['x(km)', 'y(km)', 'z(km)']
config["use_dbscan"] = True
config["use_amplitude"] = True
config["vel"] = {"p": 5.8, "s": 3.35}
config["method"] = "BGMM"
if config["method"] == "BGMM":
    config["oversample_factor"] = 10
if config["method"] == "GMM":
    config["oversample_factor"] = 10

# DBSCAN
config["bfgs_bounds"] = ((config["x(km)"][0]-1, config["x(km)"][1]+1), #x
                        (config["y(km)"][0]-1, config["y(km)"][1]+1), #y
                        (0, config["z(km)"][1]+1), #x
                        (None, None)) #t
# config["dbscan_eps"] = min(6, np.sqrt((stations["x(km)"].max()-stations["x(km)"].min())**2 +
#                                    (stations["y(km)"].max()-stations["y(km)"].min())**2)/(6.0/1.75)) #s
config["dbscan_eps"] = 50.0
config["dbscan_min_samples"] = min(4, len(stations))

# Filtering
config["min_picks_per_eq"] = 5
config["max_sigma11"] =60 #s
config["max_sigma22"] = 100.0 #m/s
config["max_sigma12"] = 1.0 #covariance

# print(config)
for k, v in config.items():
    print(f"{k}: {v}")
    

Pick format:      id                        timestamp type      prob       amp    time_idx
0  QJ01 2022-08-25 04:06:39.990000+00:00    S  0.058521  0.000665  2022-08-25
1  QJ01 2022-08-25 04:08:49.870000+00:00    P  0.374715  0.000669  2022-08-25
2  QJ01 2022-08-25 04:09:59.990000+00:00    S  0.082761  0.000009  2022-08-25
3  QJ01 2022-08-25 04:11:07.960000+00:00    P  0.202721  0.001722  2022-08-25
4  QJ01 2022-08-25 04:12:48.040000+00:00    S  0.241287  0.000145  2022-08-25
Index(['id', 'timestamp', 'type', 'prob', 'amp', 'time_idx'], dtype='object')
Station format:      id   longitude   latitude  elevation_m      x(km)      y(km)  z(km)
0  QJ01  102.925227  26.919071          863  -8.299772  13.216871  0.863
1  QJ02  102.999659  27.247152         1776  -0.037897  49.633845  1.776
2  QJ03  102.923580  27.252875         1896  -8.482575  50.269083  1.896
3  QJ04  102.724807  27.035844          838 -30.546394  26.178684  0.838
4  QJ05  102.867393  26.985724         1372 -14.719322  20.6

In [ ]:
start_time = time.time()
pbar = tqdm(sorted(list(set(picks["time_idx"]))))

event_idx0 = 0  ## current earthquake index
assignments = []

if (len(picks) > 0) and (len(picks) < 5000):
    catalogs, assignments = association(
        picks, 
        stations, 
        config,
        event_idx0,
        method=config["method"],
        pbar=pbar,
      
    )
    event_idx0 += len(catalogs)
else:
    catalogs = []
    #picks["time_idx"] = picks["timestamp"].apply(lambda x: x.strftime("%Y-%m-%dT%H")) ## process by hours
    
    for i, segment in enumerate(pbar):
        picks_ = picks[picks["time_idx"] == segment]
        if len(picks_) == 0:
            print('len(picks)==0')
            continue
        catalog, assign = association(
            picks_, 
            stations, 
            config,
            event_idx0,
            method=config["method"],
            pbar=pbar,

        )
        event_idx0 += len(catalog)
        catalogs.extend(catalog)
        assignments.extend(assign)
        print(catalog)
## 保存输出的原始文件
#print(catalogs)   


In [32]:
# os.mkdir(result_path)
catalogs = pd.DataFrame(catalogs)
catalogs.to_csv(os.path.join(result_path,'catalog_gamma_original.csv'),index=False)    # 可以不存original文件，不需要

## create catalog
catalogs = pd.DataFrame(
    catalogs, columns=["time"] + config["dims"] + ["magnitude", "sigma_time", "sigma_amp", "cov_time_amp", "event_index", "gamma_score"]
)     
# 

#catalogs["time"] = catalogs["time"].apply(lambda x: from_seconds(x))      # time换成秒数了好像
catalogs["longitude"] = catalogs["x(km)"].apply(
    lambda x: x / config["degree2km"] + config["center"][0]
)
catalogs["latitude"] = catalogs["y(km)"].apply(
    lambda x: x / config["degree2km"] + config["center"][1]
)
catalogs["depth(m)"] = catalogs["z(km)"].apply(lambda x: x * 1e3)

catalogs.sort_values(by=["time"], inplace=True)
with open(os.path.join(result_path,'gamma_catalog.csv'), 'w') as fp:
    catalogs.to_csv(
        fp,
        sep="\t",
        index=False,
        float_format="%.3f",
        date_format='%Y-%m-%dT%H:%M:%S.%f',
        columns=[
            "time",
            "magnitude",
            "longitude",
            "latitude",
            "depth(m)",
            "sigma_time",
            "sigma_amp",
            "cov_time_amp",
            "event_index",
            "gamma_score",
            "x(km)",
            "y(km)",
            "z(km)",
        ],
    )

# catalogs = catalogs[
#     ['time', 'magnitude', 'longitude', 'latitude', 'depth(m)', 'sigma_time', 'sigma_amp']
# ]
# print(catalogs)

## add assignment to picks
#assignments=pd.read_csv('D:/MyRepository/Qiaojia/AssResults_022022_032022/assignmets_gamma_original.csv',sep=",")
assignments = pd.DataFrame(assignments, columns=["pick_index", "event_index", "gamma_score"])
assignments.to_csv(os.path.join(result_path,'assignmets_gamma_original.csv'),index=False)     # 保存原始得到的文件，以防checkpoints

# picks = pd.read_csv('D:\MyRepository\Qiaojia\GammaAssociation\merged_output.csv', parse_dates=["phase_time"], usecols=["station_id", "phase_time", "phase_type", "phase_score", "phase_amplitude"])     # parse_dates用于讲某列解析为日期格式
# picks.rename(columns={"station_id": "id", "phase_time": "timestamp", "phase_type": "type", "phase_score": "prob", "phase_amplitude": "amp"}, inplace=True)
# #picks = picks[:50000]
# picks["time_idx"] = picks["timestamp"].apply(lambda x: x.strftime("%Y-%m-%d")) ## process by days
picks = picks.reset_index()  # 保留原始行号
picks = (
    picks.join(assignments.set_index("pick_index")).fillna(-1).astype({'event_index': int})
)
picks.sort_values(by=["timestamp"], inplace=True)


# result_path = 'D:/MyRepository/Qiaojia/AssResults_022022_032022'
with open(os.path.join(result_path,'gamma_picks.csv'), 'w') as fp:
    picks.to_csv(
        fp,
        sep="\t",
        index=False,
        date_format='%Y-%m-%dT%H:%M:%S.%f',
        columns=["id", "timestamp", "type", "prob", "amp", "event_index", "gamma_score"],
    )
print(f"Total time: {time.time() - start_time} seconds")

Total time: 34244.72857761383 seconds


In [ ]:
# 过滤后统计
filtered_amps = df[df['phase_amplitude'] > 1e-9]['phase_amplitude']
print(filtered_amps.describe(percentiles=[0.1, 0.5, 0.9]))

count    1.445335e+06
mean     3.602982e-04
std      9.000190e-03
min      1.019357e-09
10%      3.798645e-06
50%      9.248994e-05
90%      5.925524e-04
max      4.610873e+00
Name: phase_amplitude, dtype: float64
